In [ ]:
# get ic decay
from get_data import get_data,bars
from singletrader.processors.cs_processor import CsWinzorize,CsNeutrualize
from singletrader.shared.utility import save_pkl
from singletrader.factorlib import FactorEvaluation,summary_plot
import pandas as pd
import plotly.express as px
from plotly.figure_factory import create_table
import warnings
warnings.filterwarnings('ignore')
from singletrader.datasdk.qlib.base import MultiFactor
from singletrader.shared.utility import load_pkls
from singletrader.datasdk.sql.dataapi import get_index_cons
cs_neu = CsNeutrualize()


In [ ]:
fields = []
names = []

fields += ['$close','$open','$high','$low','$avg','$volume','$circulating_market_cap','$turnover_ratio','$money','1/$pe_ratio']
bars = ['close','open','high','low','avg','volume','circulating_market_cap','turnover_ratio','money','ep']
names += bars

In [3]:
# 获取数据
start_date = '2018-01-01'
end_date = '2023-03-31'
data = MultiFactor(name=names,field=fields,start_date=start_date,end_date=end_date)._data
index_cons = get_index_cons(start_date=start_date,end_date=end_date)
factor_data = load_pkls(r'D:\projects\singletrader_pro\samples\contribute\trademodel\predict500')
# factor_data = factor_data[index_cons.reindex(factor_data.index)['zz1000']>0]
factor_data.index = factor_data.index.set_names(['date','code'])
factor_data.name = 'lgb'
factor_data_neu = cs_neu(factor_data)
all_factor = pd.concat([factor_data,factor_data_neu],axis=1).dropna()
all_factor.columns = ['raw','neu']

{"level": "INFO", "time": "2023-04-03 16:41:47,925", "timeMillis": 1680511275824,"module": "root", "method": "python-service", "addition": "[base.py 76 _data]", "message": "开始生成因子数据..."}


In [4]:

fe = FactorEvaluation(bar_data=data,factor_data=all_factor)
fe.get_summary(groups=10,base='avg',holding_period=5,add_shift=1)


start_date:None / end_date:None


,raw,neu
ic.mean,0.0616,0.0564
ic.t-stats,10.3719,13.5811
AnnRet_Long,0.1143,0.1130
AnnRet_short,-0.3011,-0.2855
AnnRet,0.4154,0.3985
SR_Long,1.9782,2.5523
SR_Short,-5.0981,-5.8089
SR,4.4092,5.8908
TO_Long,0.1077,0.1043
TO_Short,0.1141,0.1165


In [7]:
report = fe.get_factor_detail_report(factor='raw',cost=0.00,universe=index_cons['zz500'],holding_period=5,groups=10,base='avg',total=False,add_shift=1,excess_return=True)
summary_plot(report)

In [8]:
report = fe.get_factor_detail_report(factor='neu',holding_period=5,groups=10,cost=0.0015,base='avg',universe=index_cons['zz500'],total=True,add_shift=1,excess_return=True)
summary_plot(report)